In [3]:
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Tesla T4, 15360 MiB

PyTorch: 2.10.0+cu128
CUDA available: True
GPU: Tesla T4
VRAM: 15.6 GB


In [4]:
!pip install -q boltz biopython matplotlib py3Dmol pyyaml

import boltz
print(f"✓ Boltz installed: {boltz.__version__}")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.2/399.2 kB 25.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.4/268.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.1 MB/s eta 0:00:00
   ━━

In [5]:
from google.colab import files
import os

print("=" * 60)
print("UPLOAD YOUR PROTEIN SEQUENCE FILE")
print("=" * 60)
print("\nSupported: FASTA (.fasta, .fa, .faa)")
print("Click 'Choose Files' below...\n")

uploaded = files.upload()
input_file = list(uploaded.keys())[0]
file_ext = os.path.splitext(input_file)[1].lower()

print(f"\n✓ File uploaded: {input_file}")
print(f"  Size: {os.path.getsize(input_file) / 1024:.1f} KB")
print("\nFile preview:")
!head -20 "{input_file}"

UPLOAD YOUR PROTEIN SEQUENCE FILE

Supported: FASTA (.fasta, .fa, .faa)
Click 'Choose Files' below...



Saving proteins.fasta.txt to proteins.fasta.txt

✓ File uploaded: proteins.fasta.txt
  Size: 1.3 KB

File preview:
>sp|P69905|HBA_HUMAN Hemoglobin subunit alpha OS=Homo sapiens OX=9606 GN=HBA1 PE=1 SV=2
MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGH
GKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEF
TPAVHASLDKFLASVSTVLTSKYR
>sp|P68871|HBB_HUMAN Hemoglobin subunit beta OS=Homo sapiens OX=9606 GN=HBB PE=1 SV=2
MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNP
KVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHH
FGKEFTPPVQAAYQKVVAGVANALAHKYH
>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens OX=9606 GN=INS PE=1 SV=1
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAE
DLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
>sp|P04406|G3P_HUMAN Glyceraldehyde-3-phosphate dehydrogenase OS=Homo sapiens OX=9606 GN=GAPDH PE=1 SV=3
MVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVK
AENGKLVINGKAITIFQERDPANIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVI
ISAPSADAP

In [6]:
import os
import yaml

def read_fasta(filepath):
    """Parse FASTA file into (name, sequence) tuples"""
    sequences = []
    current_name = None
    current_seq = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith('>'):
                if current_name and current_seq:
                    sequences.append((current_name, ''.join(current_seq)))
                if '|' in line:
                    parts = line[1:].split('|')
                    current_name = parts[2].split()[0] if len(parts) >= 3 else parts[0].split()[0]
                else:
                    current_name = line[1:].split()[0]
                current_seq = []
            else:
                current_seq.append(line.replace(' ', ''))
        if current_name and current_seq:
            sequences.append((current_name, ''.join(current_seq)))
    return sequences

sequences = read_fasta(input_file)
print(f"Found {len(sequences)} sequences:\n")
for name, seq in sequences:
    print(f"  ✓ {name}: {len(seq)} aa")

# Create YAML input files for Boltz (one per protein)
input_dir = "boltz_input"
os.makedirs(input_dir, exist_ok=True)

for name, seq in sequences:
    clean_name = name.replace("|", "_").replace("/", "_").replace(":", "_")
    yaml_content = {
        "version": 1,
        "sequences": [
            {
                "protein": {
                    "id": "A",
                    "sequence": seq
                }
            }
        ]
    }
    yaml_path = os.path.join(input_dir, f"{clean_name}.yaml")
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    print(f"  ✓ Created: {yaml_path}")

print(f"\n✓ {len(sequences)} YAML input files ready for Boltz-2")


Found 5 sequences:

  ✓ HBA_HUMAN: 142 aa
  ✓ HBB_HUMAN: 147 aa
  ✓ INS_HUMAN: 110 aa
  ✓ G3P_HUMAN: 333 aa
  ✓ LYSC_HUMAN: 147 aa
  ✓ Created: boltz_input/HBA_HUMAN.yaml
  ✓ Created: boltz_input/HBB_HUMAN.yaml
  ✓ Created: boltz_input/INS_HUMAN.yaml
  ✓ Created: boltz_input/G3P_HUMAN.yaml
  ✓ Created: boltz_input/LYSC_HUMAN.yaml

✓ 5 YAML input files ready for Boltz-2


In [7]:
import time

output_dir = "boltz_results"
os.makedirs(output_dir, exist_ok=True)

print("=" * 60)
print("RUNNING BOLTZ-2 STRUCTURE PREDICTION")
print("=" * 60)
print(f"Proteins to predict: {len(sequences)}")
print(f"Model: Boltz-2 (biomolecular foundation model)")
print("")
print("⚠️  First run downloads model weights (~3 GB)")
print("⚠️  Each protein takes 2-10 minutes depending on length")
print("=" * 60)

start_time = time.time()

# Run Boltz prediction on the input directory
!boltz predict "{input_dir}" \
    --out_dir "{output_dir}" \
    --use_msa_server \
    --recycling_steps 3 \
    --diffusion_samples 1

elapsed = time.time() - start_time
print(f"\n{'=' * 60}")
print(f"✓ Prediction complete in {elapsed/60:.1f} minutes")
print("=" * 60)

RUNNING BOLTZ-2 STRUCTURE PREDICTION
Proteins to predict: 5
Model: Boltz-2 (biomolecular foundation model)

⚠️  First run downloads model weights (~3 GB)
⚠️  Each protein takes 2-10 minutes depending on length
MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Extracting the CCD data to /root/.boltz/mols. This may take a bit of time. You may change the cache directory with the --cache flag.
Checking input data.
Processing 5 inputs with 2 threads.
  0% 0/5 [00:00<?, ?it/s]Generating MSA for boltz_input/HBA_HUMAN.yaml with 1 protein entities.
Calling MSA server for target HBA_HUMAN with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server
Generating MSA for boltz_input/HBB_HUMAN.yaml with 1 protein entities.
Calling MSA server for target HBB_HUMAN with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA ser

In [8]:
import glob
import json
import numpy as np

print("=" * 60)
print("BOLTZ-2 PREDICTION RESULTS")
print("=" * 60)

# Show directory structure
print("\nOutput directory structure:")
!find {output_dir} -type f | sort | head -40

# Find structure files
cif_files = sorted(glob.glob(f"{output_dir}/**/*.cif", recursive=True))
pdb_files_out = sorted(glob.glob(f"{output_dir}/**/*.pdb", recursive=True))
all_structures = cif_files + pdb_files_out

print(f"\nStructure files found: {len(all_structures)}")
for sf in all_structures:
    print(f"  📄 {os.path.basename(sf)} ({os.path.getsize(sf)/1024:.1f} KB)")

# Find and display confidence scores
json_files = sorted(glob.glob(f"{output_dir}/**/*confidence*.json", recursive=True))
if not json_files:
    json_files = sorted(glob.glob(f"{output_dir}/**/*.json", recursive=True))

print(f"\nConfidence data files: {len(json_files)}")
for jf in json_files:
    print(f"\n  📊 {os.path.basename(jf)}")
    try:
        with open(jf, 'r') as f:
            scores = json.load(f)
        for key, val in scores.items():
            if isinstance(val, (int, float)):
                print(f"     {key}: {val:.4f}")
            elif isinstance(val, list) and val and isinstance(val[0], (int, float)):
                arr = np.array(val)
                print(f"     {key}: mean={arr.mean():.2f}, min={arr.min():.2f}, max={arr.max():.2f}")
    except Exception as e:
        print(f"     Could not parse: {e}")


BOLTZ-2 PREDICTION RESULTS

Output directory structure:
boltz_results/boltz_results_boltz_input/lightning_logs/version_0/events.out.tfevents.1772039432.c209e9e436e0.1293.0
boltz_results/boltz_results_boltz_input/lightning_logs/version_0/hparams.yaml
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_0.csv
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_unpaired_tmp_env/bfd.mgnify30.metaeuk30.smag30.a3m
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_unpaired_tmp_env/msa.sh
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_unpaired_tmp_env/out.tar.gz
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_unpaired_tmp_env/pdb70.m8
boltz_results/boltz_results_boltz_input/msa/G3P_HUMAN_unpaired_tmp_env/uniref.a3m
boltz_results/boltz_results_boltz_input/msa/HBA_HUMAN_0.csv
boltz_results/boltz_results_boltz_input/msa/HBA_HUMAN_unpaired_tmp_env/bfd.mgnify30.metaeuk30.smag30.a3m
boltz_results/boltz_results_boltz_input/msa/HBA_HUMAN_unpaired_tmp_env/msa.sh
boltz_results/boltz_

In [9]:
import py3Dmol

print("=" * 60)
print("3D STRUCTURE VISUALIZATION")
print("=" * 60)

for sf in all_structures[:5]:
    name = os.path.basename(sf)
    print(f"\n🔬 {name}")

    with open(sf, 'r') as f:
        structure_data = f.read()

    fmt = "cif" if sf.endswith(".cif") else "pdb"
    view = py3Dmol.view(width=800, height=500)
    view.addModel(structure_data, fmt)
    view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'rwb', 'min': 50, 'max': 90}}})
    view.zoomTo()
    view.show()


3D STRUCTURE VISUALIZATION

🔬 G3P_HUMAN_model_0.cif


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


🔬 HBA_HUMAN_model_0.cif


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


🔬 HBB_HUMAN_model_0.cif


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


🔬 INS_HUMAN_model_0.cif


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


🔬 LYSC_HUMAN_model_0.cif


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
# Copy input
!cp "{input_file}" "{output_dir}/"

with open(f"{output_dir}/analysis_summary.txt", "w") as f:
    f.write("=" * 70 + "\n")
    f.write("BOLTZ-2 STRUCTURE PREDICTION - SUMMARY\n")
    f.write("=" * 70 + "\n\n")
    f.write(f"Input file: {input_file}\n")
    f.write(f"Proteins analyzed: {len(sequences)}\n")
    f.write(f"Model: Boltz-2\n")
    f.write(f"Processing time: {elapsed/60:.1f} minutes\n\n")
    f.write("PROTEINS:\n")
    f.write("-" * 70 + "\n")
    for i, (name, seq) in enumerate(sequences, 1):
        f.write(f"{i}. {name} ({len(seq)} aa)\n")
        f.write(f"   Sequence: {seq[:60]}...\n\n")
    f.write("OUTPUT FILES:\n")
    f.write("-" * 70 + "\n")
    for sf in all_structures:
        f.write(f"  • {sf}\n")

print("✓ Summary created")

!zip -r -q boltz_results.zip {output_dir}/
print("\nPackage size:")
!ls -lh boltz_results.zip

print("\nDownloading...")
files.download('boltz_results.zip')
print("\n✓ Done! Move results to stowers-gpu-tools/boltz/results/sample_output/")


✓ Summary created

Package size:
-rw-r--r-- 1 root root 14M Feb 25 17:15 boltz_results.zip

Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Done! Move results to stowers-gpu-tools/boltz/results/sample_output/
